One match with embedding:

In [64]:
import os
import re
import time
import pandas as pd
from langchain import hub
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Set environment variable for OpenAI API Key
os.environ["OPENAI_API_KEY"] = 'YOURAPIKEY'

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def parse_summary(summary):
    if summary.isdigit():
        return {'matchday': int(summary), 'success': None, 'action_type': None,
                'zone_from': None, 'player_from': None, 'player_to': None,
                'zone_to': None, 'team': None, 'minute': int(summary)}
    
    pattern = (
        r"Matchday (\d+): (Successful|Unsuccessful) (\w+) in zone (\w+) by player ([\w\s]+?)"
        r"(?: to player ([\w\s]+?) in zone (\w+))? for team ([\w\s]+) at minute (\d+):(\d+)"
    )

    match = re.search(pattern, summary)
    if match:
        matchday, success, action_type, zone_from, player_from, player_to, zone_to, team, minute, second = match.groups()
        player_from = player_from.strip()
        player_to = player_to.strip() if player_to else "N/A"
        zone_to = zone_to.strip() if zone_to else zone_from
        team = team.strip()

        assist = None
        
        return {
            'matchday': int(matchday),
            'success': success,
            'action_type': action_type,
            'zone_from': zone_from,
            'player_from': player_from,
            'player_to': player_to,
            'zone_to': zone_to,
            'team': team,
            'minute': int(minute),
            'assist': assist
        }
    return None

# CSV Loader
class CSVLoader:
    def __init__(self, filepath):
        self.filepath = filepath

    def load(self):
        try:
            return pd.read_csv(self.filepath, dtype=str)
        except Exception as e:
            print(f"Failed to load CSV: {e}")
            return None

# Initialize and load documents
loader = CSVLoader("csv-summaries/f24-100-2022-2288345-eventdetails.csv")
docs = loader.load()
docs = docs.astype(str)

# Filter out None values after parsing summaries
parsed_docs = [doc for doc in (parse_summary(summary) for summary in docs['Summary'] if pd.notna(summary)) if doc is not None]

# Implementing the assist identification with a safeguard against None entries
def identify_assists(actions):
    for i in range(len(actions) - 1):
        current_action = actions[i]
        next_action = actions[i + 1]
        
        if current_action and next_action:
            if (current_action['action_type'] == 'Pass' and
                next_action['action_type'] in ['Shot on target', 'Shot on post', 'Shot saved', 'Goal'] and
                current_action['team'] == next_action['team'] and
                abs(next_action['minute'] - current_action['minute']) <= 1):
                current_action['assist'] = True

    return actions

parsed_docs = identify_assists(parsed_docs)

if docs is None or not isinstance(docs, pd.DataFrame):
    print("Failed to load or invalid document format.")
else:
    parsed_docs = [parse_summary(doc) for doc in docs['Summary'] if pd.notna(doc)]
    texts = [f"{doc['action_type']}: {'successful' if doc['success'] == 'Successful' else 'unsuccessful'} in minute {doc['minute']} by {doc['player_from']} in zone {doc['zone_from']} to {doc['player_to']} in zone {doc['zone_to']} for team {doc['team']}" for doc in parsed_docs if doc]
    texts = [remove_stopwords(text) for text in texts]
    ids = [str(i) for i in range(len(texts))]

    # Initialize LangChain components
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
    vectorstore = Chroma.from_texts(texts=texts, ids=ids, embedding=OpenAIEmbeddings(model="text-embedding-3-small"), persist_directory="vectorstore")
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 150})

    # Define modular prompt templates
    introduction_prompt = """
    - You are a football analyst. Your task is to provide analysis based on the event data provided.
    - Define "successful pass" as a pass that reaches another player of the same team without interruption.
    - Each row in the document represents one event. Only count the events that are explicitly asked for in the query.
    """

    event_counting_prompt = """
    - Define "successful pass" as a pass that reaches another player of the same team without interruption.
    - When I ask for passes, only count the passes.
    - A shot is identified by 'Shot on target', 'Shot on post', 'Shot saved' and 'Goal' in the summaries.
    - Count all shot events when I explicitly ask for shots. Not only goals.
    - Defensive actions is defined by 'Tackle', 'Interception' and 'Clearance'.
    """

    match_result_prompt = """
    - When I ask for the result, tell me the result of the match based on how many goals each team scored.
    - The team that won the match is the team that scored the most goals.
    - If they scored equally many goals, it is a draw.
    """

    assists_chances_prompt = """
    - The assist is the pass in the event just before a shot. Count all shot events. Not only goals.
    - When I ask for chances created, I mean assists.
    """

    context_question_prompt = """
    {context}

    Question: {question}

    Helpful Answer:
    """

    # Combine modular prompts into a complete prompt template
    complete_prompt_template = (
    introduction_prompt +
    event_counting_prompt +
    match_result_prompt +
    assists_chances_prompt +
    context_question_prompt
    )

    custom_rag_prompt = PromptTemplate.from_template(complete_prompt_template)

    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | custom_rag_prompt
        | llm
        | StrOutputParser()
    )

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jesperpilegaard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
# Function to ask multiple questions and store responses in a CSV
def ask_multiple_questions_and_save(questions, chain, output_csv):
    results = []
    for question in questions:
        start_time = time.time()
        answer = chain.invoke(question)
        end_time = time.time()
        response_time = end_time - start_time
        results.append({
            'Question': question,
            'Answer': answer,
            'Response Time (s)': response_time
        })
    
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Responses saved to {output_csv}")

questions = [
    "How many successful passes did each team have?",
    "How many unsuccessful passes were made by each team?",
    "Which team had the most offside passes?",
    "How many fouls were committed by each team?",
    "Which team was awarded the most corners?",
    "How many shots did each team have?",
    "Which team created the most chances?",
    "What was the result of the match?",
    "How many successful defensive actions did each team have?",
    "Which players had most successful passes between each other?",
    "Which players scored the goals?",
    "When was the goals scored?",
    "In which zones were the goals scored for each team?",
    "In which zones were the chances created for each team?",
    "Which players created the chances for each team?",
    "How many goals were scored in the first half of the match?",
    "How many goals were scored in the second half of the match?",
    "In a time period of 15 minutes, when was the most shots taken?",
    "How did Viborg FF perform?",
    "How did Aalborg BK perform?",
    "Which player was most involved in the events?",
    "Which player had most shots?",
    "In which half were the most chances created?",
    "In which zone did Jeppe Grønning have most actions?"
]

# Ask the questions and save the responses
ask_multiple_questions_and_save(questions, rag_chain, "responses/query_results_gpt-3.5_embedding_150.csv")

Responses saved to responses/query_results_gpt-3.5_embedding_150.csv


One match without embedding:

In [21]:
import time
import os
import re
import pandas as pd
from langchain import hub
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable

# Set environment variable for OpenAI API Key
os.environ["OPENAI_API_KEY"] = 'YOURAPIKEY'


class QueryContextRunnable(Runnable):
    def __init__(self, context):
        self.context = context

    def invoke(self, question, config=None):  # Add 'config' parameter to handle additional configuration
        return {"context": self.context, "question": question}

class EventDetailLoader:
    def __init__(self, filepath):
        self.filepath = filepath

    def load_csv(self):
        try:
            return pd.read_csv(self.filepath, dtype=str)
        except Exception as e:
            print(f"Error loading CSV: {e}")
            return None

class EventParser:
    @staticmethod
    def parse_summary(summary):
        pattern = (
            r"Matchday (\d+): (Successful|Unsuccessful) ([\w\s]+?) in zone (\w+) by player ([\w\s]+?)"
            r"(?: to player ([\w\s]+?) in zone (\w+))? for team ([\w\s]+) at minute (\d+):(\d+)"
        )
        match = re.search(pattern, summary)
        if match:
            action_type = match.group(3)
            return {
                'matchday': int(match.group(1)),
                'success': match.group(2),
                'action_type': action_type,
                'zone_from': match.group(4),
                'player_from': match.group(5).strip(),
                'player_to': match.group(6).strip() if match.group(6) else "N/A",
                'zone_to': match.group(7).strip() if match.group(7) else match.group(4),
                'team': match.group(8).strip(),
                'minute': int(match.group(9)),
                'assist': None  # Initially, assist is None
            }
        return None

    @staticmethod
    def identify_assists(docs):
        for i in range(len(docs) - 1):
            current_action = docs[i]
            next_action = docs[i + 1]
            if current_action['action_type'] == 'Pass' and \
               next_action['action_type'].startswith('Shot') and \
               current_action['team'] == next_action['team'] and \
               abs(next_action['minute'] - current_action['minute']) <= 1:
                next_action['assist'] = current_action['player_from']
        return docs

def format_docs(docs):
    detail_dict = {}
    for doc in docs:
        team = doc['team']
        action_type = doc['action_type']
        success_status = 'Successful' if doc['success'] == 'Successful' else 'Unsuccessful'
        
        if team not in detail_dict:
            detail_dict[team] = {}
        
        if action_type not in detail_dict[team]:
            detail_dict[team][action_type] = {'Successful': [], 'Unsuccessful': []}
        
        detail_dict[team][action_type][success_status].append(doc)

    detailed_summary = []
    for team, actions in detail_dict.items():
        detailed_summary.append(f"Team: {team}")
        for action, entries in actions.items():
            for status, events in entries.items():
                count = len(events)
                if count > 0:
                    detailed_summary.append(f"  {count} {status} {action} events:")
                    for entry in events[:10]:
                        assist_info = f" with assist by {entry['assist']}" if 'assist' in entry and entry['assist'] else ""
                        detailed_summary.append(
                            f"   - {entry['minute']}' by {entry['player_from']} to {entry['player_to']} in zone {entry['zone_from']} to {entry['zone_to']}{assist_info}"
                        )
                    if count > 10:
                        detailed_summary.append(f"   ...and {count - 10} more events.")
    return "\n".join(detailed_summary)

# Modify the main execution script to use the new formatting function
if __name__ == "__main__":
    loader = EventDetailLoader("csv-summaries/f24-100-2022-2288345-eventdetails.csv")
    docs = loader.load_csv()
    if docs is not None:
        parsed_docs = [EventParser.parse_summary(summary) for summary in docs['Summary'] if pd.notna(summary)]
        parsed_docs = [doc for doc in parsed_docs if doc is not None]
        parsed_docs = EventParser.identify_assists(parsed_docs)

        formatted_docs = format_docs(parsed_docs)
        
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

        # Define modular prompt templates
        introduction_prompt = """
        - You are a football analyst. Your task is to provide analysis based on the event data provided.
        - Define "successful pass" as a pass that reaches another player of the same team without interruption.
        - Each row in the document represents one event. Only count the events that are explicitly asked for in the query.
        - Only respond to what explicitely is asked for.
        """

        # event_counting_prompt = """
        # - Define "successful pass" as a pass that reaches another player of the same team without interruption.
        # - When I ask for passes, only count the passes.
        # - A shot is identified by 'Shot on target', 'Shot on post', 'Shot saved' and 'Goal' in the summaries.
        # - Count all shot events when I explicitly ask for shots. Not only goals.
        # - Defensive actions is defined by 'Tackle', 'Interception' and 'Clearance'.
        # """

        # match_result_prompt = """
        # - When I ask for the result, tell me the result of the match based on how many goals each team scored.
        # - The team that won the match is the team that scored the most goals.
        # - If they scored equally many goals, it is a draw.
        # """

        # assists_chances_prompt = """
        # - The assist is the pass in the event just before a shot. Count all shot events. Not only goals.
        # - When I ask for chances created, I mean assists.
        # """

        context_question_prompt = """
        {context}

        Question: {question}

        Helpful Answer:
        """


        # Combine modular prompts into a complete prompt template
        complete_prompt_template = (
        introduction_prompt +
        event_counting_prompt +
        match_result_prompt +
        assists_chances_prompt +
        context_question_prompt
        )

        custom_prompt = PromptTemplate.from_template(introduction_prompt + context_question_prompt)

        query_context = QueryContextRunnable(formatted_docs)
        response_chain = (query_context | custom_prompt | llm | StrOutputParser())

In [22]:
questions = [
    "How many successful passes did each team have?",
    "How many unsuccessful passes were made by each team?",
    "Which team had the most offside passes?",
    "How many fouls were committed by each team?",
    "Which team was awarded the most corners?",
    "How many shots did each team have?",
    "Which team created the most chances?",
    "What was the result of the match?",
    "How many successful defensive actions did each team have?",
    "Which players had most successful passes between each other?",
    "Which players scored the goals?",
    "When was the goals scored?",
    "In which zones were the goals scored for each team?",
    "In which zones were the chances created for each team?",
    "Which players created the chances for each team?",
    "How many goals were scored in the first half of the match?",
    "How many goals were scored in the second half of the match?",
    "In a time period of 15 minutes, when was the most shots taken?",
    "How did Aalborg BK perform?",
    "How did Viborg FF perform?",
    "Which player was most involved in the events?",
    "Which player had most shots?",
    "In which half were the most chances created?",
    "In which zone did Jeppe Grønning have most actions?"
]

results = []
for question in questions:
    start_time = time.time()
    response = response_chain.invoke(question)
    end_time = time.time()
    response_time = end_time - start_time
    results.append({
        'Question': question,
        'Answer': response,  # Ensure we handle the response correctly
        'Response Time (s)': response_time
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)
        
# Save results to a CSV file
results_df.to_csv('responses/query_results_gpt-3.5_noprompt.csv', index=False)
        
# Display the DataFrame
print(results_df)

                                             Question  \
0      How many successful passes did each team have?   
1   How many unsuccessful passes were made by each...   
2             Which team had the most offside passes?   
3         How many fouls were committed by each team?   
4            Which team was awarded the most corners?   
5                  How many shots did each team have?   
6                Which team created the most chances?   
7                   What was the result of the match?   
8   How many successful defensive actions did each...   
9   Which players had most successful passes betwe...   
10                    Which players scored the goals?   
11                         When was the goals scored?   
12  In which zones were the goals scored for each ...   
13  In which zones were the chances created for ea...   
14   Which players created the chances for each team?   
15  How many goals were scored in the first half o...   
16  How many goals were scored 

Multiple matches:

In [33]:
import os
import re
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable
from langchain_community.llms import Ollama

# Set environment variable for OpenAI API Key
os.environ["OPENAI_API_KEY"] = 'YOURAPIKEY'


class QueryContextRunnable(Runnable):
    def __init__(self, context):
        self.context = context

    def invoke(self, question, config=None):
        return {"context": self.context, "question": question}


class EventDetailLoader:
    def __init__(self, directory):
        self.directory = directory

    def load_csvs(self, limit=18):
        events = []
        csv_files = sorted([f for f in os.listdir(self.directory) if f.endswith('.csv')])[:limit]
        
        # Print filenames to verify multiple files are being loaded
        print(f"Loading files: {csv_files}")
        
        for filename in csv_files:
            filepath = os.path.join(self.directory, filename)
            try:
                df = pd.read_csv(filepath, dtype=str)
                # Print some rows to verify content
                print(f"Loaded file {filename} with {len(df)} rows")
                print(df.head())
                events.extend(df['Summary'].dropna().tolist())
            except Exception as e:
                print(f"Error loading CSV {filename}: {e}")
        return events


class EventParser:
    @staticmethod
    def parse_summary(summary):
        pattern = (
            r"Matchday (\d+): (Successful|Unsuccessful) ([\w\s]+?) in zone (\w+) by player ([\w\s]+?)"
            r"(?: to player ([\w\s]+?) in zone (\w+))? for team ([\w\s]+) at minute (\d+):(\d+)"
        )
        match = re.search(pattern, summary)
        if match:
            action_type = match.group(3)
            if action_type.lower() == "pass":
                return None  # Skip passes to reduce token usage
            return {
                'matchday': int(match.group(1)),
                'success': match.group(2),
                'action_type': action_type,
                'zone_from': match.group(4),
                'player_from': match.group(5).strip(),
                'player_to': match.group(6).strip() if match.group(6) else "N/A",
                'zone_to': match.group(7).strip() if match.group(7) else match.group(4),
                'team': match.group(8).strip(),
                'minute': int(match.group(9)),
                'assist': None  # Initially, assist is None
            }
        return None

    @staticmethod
    def identify_assists(docs):
        for i in range(len(docs) - 1):
            current_action = docs[i]
            next_action = docs[i + 1]
            if current_action['action_type'] == 'Pass' and \
               next_action['action_type'].startswith('Shot') and \
               current_action['team'] == next_action['team'] and \
               abs(next_action['minute'] - current_action['minute']) <= 1:
                next_action['assist'] = current_action['player_from']
        return docs


def format_docs(docs):
    summary = {}
    for doc in docs:
        team = doc['team']
        matchday = doc['matchday']
        action_type = doc['action_type']
        success_status = 'Successful' if doc['success'] == 'Successful' else 'Unsuccessful'

        if team not in summary:
            summary[team] = {}

        if matchday not in summary[team]:
            summary[team][matchday] = {}

        if action_type not in summary[team][matchday]:
            summary[team][matchday][action_type] = {'Successful': 0, 'Unsuccessful': 0}

        summary[team][matchday][action_type][success_status] += 1

    detailed_summary = []
    for team, matchdays in summary.items():
        detailed_summary.append(f"Team: {team}")
        for matchday, actions in matchdays.items():
            detailed_summary.append(f"  Matchday: {matchday}")
            for action, counts in actions.items():
                for status, count in counts.items():
                    if count > 0:
                        detailed_summary.append(f"    {count} {status} {action} events")
    return "\n".join(detailed_summary)


if __name__ == "__main__":
    loader = EventDetailLoader("/Users/jesperpilegaard/Desktop/Superliga 2022-2023/csv-summaries")
    all_docs = loader.load_csvs()

    parsed_docs = []
    for summary in all_docs:
        parsed_summary = EventParser.parse_summary(summary)
        if parsed_summary is not None:
            parsed_docs.append(parsed_summary)

    parsed_docs = EventParser.identify_assists(parsed_docs)
    formatted_docs = format_docs(parsed_docs)

    llm = ChatOpenAI(model="gpt-4o", temperature=0.1)

    # Define modular prompt templates
    introduction_prompt = """
    - You are a football analyst. Your task is to provide analysis based on the event data provided.
    - Each CSV file represents one match. The teams within the same CSV file are considered the as opponents in that match.
    - Each row in the documents represents one event. Only count the events that are explicitly asked for in the query.
    - Only respond to what explicitely is asked for.
    """

    # event_counting_prompt = """
    # - A shot is identified by 'Shot on target', 'Shot on post', 'Shot saved' and 'Goal' in the summaries.
    # - Count all shot events when I explicitly ask for shots. Not only goals.
    # - Defensive actions is defined by 'Tackle', 'Interception' and 'Clearance'.
    # """

    # match_result_prompt = """
    # - When I ask for the result, tell me the result of the match based on how many goals each team scored.
    # - The team that won the match is the team that scored the most goals.
    # - If they scored equally many goals, it is a draw.
    # """

    # assists_chances_prompt = """
    # - The assist is the pass in the event just before a shot. Count all shot events. Not only goals.
    # - When I ask for chances created, I mean assists.
    # """

    context_question_prompt = """
    {context}

    Question: {question}

    Helpful Answer:
    """

    # Combine modular prompts into a complete prompt template
    complete_prompt_template = (
        introduction_prompt +
        event_counting_prompt +
        match_result_prompt +
        assists_chances_prompt +
        context_question_prompt
    )

    custom_rag_prompt = PromptTemplate.from_template(complete_prompt_template)

    query_context = QueryContextRunnable(formatted_docs)
    response_chain = (query_context | custom_rag_prompt | llm | StrOutputParser())


Loading files: ['f24-100-2022-2288344-eventdetails.csv', 'f24-100-2022-2288345-eventdetails.csv', 'f24-100-2022-2288346-eventdetails.csv', 'f24-100-2022-2288347-eventdetails.csv', 'f24-100-2022-2288348-eventdetails.csv', 'f24-100-2022-2288349-eventdetails.csv', 'f24-100-2022-2288350-eventdetails.csv', 'f24-100-2022-2288351-eventdetails.csv', 'f24-100-2022-2288352-eventdetails.csv', 'f24-100-2022-2288353-eventdetails.csv', 'f24-100-2022-2288354-eventdetails.csv', 'f24-100-2022-2288355-eventdetails.csv', 'f24-100-2022-2290007-eventdetails.csv', 'f24-100-2022-2290008-eventdetails.csv', 'f24-100-2022-2290009-eventdetails.csv', 'f24-100-2022-2290010-eventdetails.csv', 'f24-100-2022-2290011-eventdetails.csv', 'f24-100-2022-2290012-eventdetails.csv']
Loaded file f24-100-2022-2288344-eventdetails.csv with 1169 rows
                                             Summary
0  Matchday 1: Successful Pass in zone 3C by play...
1  Matchday 1: Unsuccessful Pass in zone 3C by pl...
2  Matchday 1: Unsucce

In [34]:
questions = [
    "Which team has scored most goals from Matchday 1 to Matchday 3?",
    "Which player has had most shots from Matchday 1 to Matchday 3?",
    "Which team has most saves from Matchday 1 to Matchday 3?",
    "Which player has most interceptions from Matchday 1 to Matchday 3?",
    "Which team has most corners awarded from Matchday 1 to Matchday 3?",
    "Which player received most cards from Matchday 1 to Matchday 3?",
    "Which 10 players received most cards from Matchday 1 to Matchday 3?",
    "Which teams played each other in Matchday 1?",
    "How did FC Midtjylland perform from Matchday 1 to Matchday 3?",
    "Which team had the most offside passes from Matchday 1 to Matchday 3?",
    "Which team had the most successful passes from Matchday 1 to Matchday 3?",
    "How many chances did Odense Boldklub create from Matchday 1 to Matchday 3?",
    "What was the result of Brøndby IF's matches from Matchday 1 to Matchday 3?"
]

results = []
for question in questions:
    start_time = time.time()
    response = response_chain.invoke(question)
    end_time = time.time()
    response_time = end_time - start_time
    results.append({
        'Question': question,
        'Answer': response,  # Ensure we handle the response correctly
        'Response Time (s)': response_time
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)
        
# Save results to a CSV file
results_df.to_csv('responses/query_results_gpt-4o_multi_noprompt.csv', index=False)
        
# Display the DataFrame
print(results_df)

                                             Question  \
0   Which team has scored most goals from Matchday...   
1   Which player has had most shots from Matchday ...   
2   Which team has most saves from Matchday 1 to M...   
3   Which player has most interceptions from Match...   
4   Which team has most corners awarded from Match...   
5   Which player received most cards from Matchday...   
6   Which 10 players received most cards from Matc...   
7        Which teams played each other in Matchday 1?   
8   How did FC Midtjylland perform from Matchday 1...   
9   Which team had the most offside passes from Ma...   
10  Which team had the most successful passes from...   
11  How many chances did Odense Boldklub create fr...   
12  What was the result of Brøndby IF's matches fr...   

                                               Answer  Response Time (s)  
0   The team that has scored the most goals from M...          11.627017  
1   Based on the event data provided, the player w.